In [1]:
#required dependencies
import os
from pathlib import Path
import numpy as np
from matplotlib.image import imread
import matplotlib.pyplot as plt


In [2]:

def covariance_matrix(matrix):
    # the columns are each of the measurement types
    # rows are each of the measurements

    
    # take the mean along the columns for each of the measurement types
    normalized_matrix = (matrix - np.mean(matrix)).T 
    
    # compute the covariance matrix
    covariance_matrix = np.matmul(normalized_matrix, normalized_matrix.T) / (normalized_matrix.shape[0]-1)
    return covariance_matrix


In [3]:
    
def slice_eigen(color_matrix, num_components):
    
    # solve for the eigenvectors and eigenvalues
    evalues, evectors = np.linalg.eigh(covariance_matrix(color_matrix))
    if (num_components > len(evalues)):
        num_components = len(evalues)
        
    # slice the desired principal components
    desired_index = np.argsort(evalues)[::-1][:num_components]
    sorted_evalues = evalues[desired_index]
    sorted_evectors = evectors[:,desired_index]
    return sorted_evalues, sorted_evectors


In [4]:

def build_color_matrix(color_matrix, num_components):
    reval, revectors = slice_eigen(color_matrix, num_components)
    
    # compute the projection
    res = np.dot(revectors.T, (color_matrix - np.mean(color_matrix)).T)
    resul = np.dot(revectors, res)
    
    # need to reverse normalize the color_matrix so that it is centered on the mean of the original matrix
    resultt = (resul + np.mean(color_matrix)).T 
    
    # clean the data so that it is integers
    final = np.uint8(np.absolute(resultt)) 
    return final


In [5]:

def create_new_image(file_name, num_components):
    # read image data into a matrix
    test_directory = 'test_files/'
    raw = imread(test_directory+file_name)
    
    # extract the RGB bytes
    r = raw[:, :, 0]
    g = raw[:, :, 1]
    b = raw[:, :, 2]
    
    # build each matrix using the functions defined above
    r_new = build_color_matrix(r, num_components)
    g_new = build_color_matrix(g, num_components)
    b_new = build_color_matrix(b, num_components)
    
    # stack the color matrices
    new_image = np.dstack((r_new, g_new, b_new))
    
    # new file name and saving the file
    new_file_name = file_name.split('.')[0] + '_' + str(num_components) + '_result.' + file_name.split('.')[1]
    result_directory = 'result_files/'
    plt.imsave(result_directory + new_file_name, new_image)
    

In [6]:

# if the result_files folder does not exist, create it
if not Path('result_files').is_dir():
      os.makedirs('result_files')
        
# Choose your image and number of components
# place the image in the test_files folder

num_components = 100

# use just the file name, not the whole path name

create_new_image("eiffel.jpg", num_components)
